In [1]:
import pandas as pd

run = pd.read_pickle("evaluations/testdev_ft_clipbase_owllarge/results_snapshot.pkl")
run_with_skips = run
run = run[~run["skipped"]]

In [2]:
import re 

for i, row in run.iterrows():
    with open(f"../data/encoded_questions/{row['question_id']}.lp") as f:
        question_encoding = f.read()
    
    run.loc[i, "has_select"] = bool(re.search("select\(", question_encoding))
    run.loc[i, "has_filter"] = bool(re.search("filter\(", question_encoding))
    run.loc[i, "has_filter_any"] = bool(re.search("filter_any\(", question_encoding))
    run.loc[i, "has_relate"] = bool(re.search("relate\(", question_encoding)) 
    run.loc[i, "has_relate_any"] = bool(re.search("relate_any\(", question_encoding))
    run.loc[i, "has_relate_attr"] = bool(re.search("relate_attr\(", question_encoding))
    run.loc[i, "has_compare"] = bool(re.search("compare\(", question_encoding))
    run.loc[i, "has_query_attr"] = bool(re.search("query\(\w+, \w+, name\)", question_encoding))
    run.loc[i, "has_query_name"] = bool(re.search("query\(\w+, \w+, (?!name)\w+\)", question_encoding))
    run.loc[i, "has_verify_attr"] = bool(re.search("verify_attr\(", question_encoding))
    run.loc[i, "has_verify_rel"] = bool(re.search("verify_rel\(", question_encoding))
    run.loc[i, "has_choose_attr"] = bool(re.search("choose_attr\(", question_encoding))
    run.loc[i, "has_choose_rel"] = bool(re.search("choose_rel\(", question_encoding))
    
    run.loc[i, "has_exist"] = bool(re.search("exist\(", question_encoding))
    run.loc[i, "has_two_same"] = bool(re.search("two_same\(", question_encoding))
    run.loc[i, "has_two_different"] = bool(re.search("two_different\(", question_encoding))
    run.loc[i, "has_all_same"] = bool(re.search("all_same\(", question_encoding))
    run.loc[i, "has_all_different"] = bool(re.search("all_different\(", question_encoding))
    run.loc[i, "has_common"] = bool(re.search("common\(", question_encoding))
    run.loc[i, "has_and"] = bool(re.search("and\(", question_encoding))
    run.loc[i, "has_or"] = bool(re.search("or\(", question_encoding))

/tmp/ipykernel_1132731/3510796415.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run.loc[i, "has_select"] = bool(re.search("select\(", question_encoding))
/tmp/ipykernel_1132731/3510796415.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run.loc[i, "has_filter"] = bool(re.search("filter\(", question_encoding))
/tmp/ipykernel_1132731/3510796415.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [3]:
op_types = [t[4:] for t in list(run) if t.startswith("has_")]
percentages = []

# question type
percentages.append({
    "name": "all",
    "correct": run[run["correct"]].shape[0] / run.shape[0]
})
binary = run[(run["answer"] == "no") | (run["answer"] == "yes")]
percentages.append({
    "name": "binary",
    "correct": binary[binary["correct"]].shape[0] / binary.shape[0]
})
open = run[(run["answer"] != "no") & (run["answer"] != "yes")]
percentages.append({
    "name": "open",
    "correct": open[open["correct"]].shape[0] / open.shape[0]
})

# operations
for op in op_types:
    percentages.append({
        "name": op,
        "correct": run[run[f"has_{op}"] & run["correct"]].shape[0] / max(run[run[f"has_{op}"]].shape[0], 1)
    })
    
results = pd.DataFrame(percentages)
results["correct"] = results["correct"] * 100
print(results.style.format(precision=2).hide(axis="index").to_latex())


\begin{tabular}{lr}
name & correct \\
all & 41.46 \\
binary & 57.76 \\
open & 32.23 \\
select & 41.46 \\
filter & 43.71 \\
filter_any & 40.22 \\
relate & 38.58 \\
relate_any & 7.61 \\
relate_attr & 24.00 \\
compare & 30.43 \\
query_attr & 23.61 \\
query_name & 38.51 \\
verify_attr & 61.46 \\
verify_rel & 55.66 \\
choose_attr & 65.70 \\
choose_rel & 47.87 \\
exist & 58.01 \\
two_same & 42.48 \\
two_different & 40.58 \\
all_same & 80.95 \\
all_different & 60.00 \\
common & 14.58 \\
and & 56.46 \\
or & 56.03 \\
\end{tabular}



In [4]:
run[run["has_relate_any"]][["question_id", "semantic_str", "answer", "model_response"]]

,question_id,semantic_str,answer,model_response
6,20567512,"select: airplane (11)->relate: _,flying above,...",ocean,[moon]
9,201079951,select: window (0)->filter: open [0]->relate: ...,drapes,[mirror]
10,201079952,"select: window (0)->relate: _,around,s (12) [0...",drapes,[mirror]
13,202218649,"select: chalkboard (0)->relate: _,hanging abov...",picture,[toaster_oven]
24,20982539,select: person (6)->filter fatness: skinny [0]...,table,[sweater]
...,...,...,...,...
12537,201935187,"select: building (11)->relate: lamp,in front o...",tree,[dining_table]
12557,201935363,"select: camera (9)->relate: woman,to the right...",bricks,[dining_table]
12559,202040263,"select: platform (4)->relate: train,near,s (0)...",train station,[handbag]
12566,201535906,"select: rope (7)->relate: _,attached to,o (10)...",coat,UNSAT


In [25]:
run[run["correct"]].shape[0] / run.shape[0]

0.40289121202221495

In [5]:
run["runtime_sec"].mean()

0.9938848002138715

In [10]:
run["model_response"] = run["model_response"].str[0]

/tmp/ipykernel_466951/2138938923.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run["model_response"] = run["model_response"].str[0]


In [5]:
run.to_csv("testdev_base_clipbase_owllarge.csv")